## Clean up Uber/Lyft data 

blatantly cribbed from Matt's video (https://www.youtube.com/watch?v=6EozEvfTcBI)

### Setup the cluster

In [1]:
import dask.dataframe as dd
from distributed import Client
from coiled import Cluster

import numpy as np
import pandas as pd

cluster = Cluster(
    account="dask-engineering",
    n_workers=12,
    package_sync=True,
    backend_options={"region": "us-east-1"},
    worker_memory="64 GiB",
)
client = Client(cluster)
client

  Cloning https://github.com/dask/dask.git (to revision 0d8e12be4c2261b3457978c16aba7e893b1cf4a1) to /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-req-build-0z792_g2
  Cloning https://github.com/dask/distributed.git (to revision f83025935383d13033fe0dcd4af2ee689078cb40) to /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-req-build-_621il8v


  Running command git clone --filter=blob:none --quiet https://github.com/dask/distributed.git /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-req-build-_621il8v
  Running command git clone --filter=blob:none --quiet https://github.com/dask/dask.git /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-req-build-0z792_g2
  Running command git rev-parse -q --verify 'sha^f83025935383d13033fe0dcd4af2ee689078cb40'
  Running command git fetch -q https://github.com/dask/distributed.git f83025935383d13033fe0dcd4af2ee689078cb40


  Resolved https://github.com/dask/distributed.git to commit f83025935383d13033fe0dcd4af2ee689078cb40
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started


  Running command git rev-parse -q --verify 'sha^0d8e12be4c2261b3457978c16aba7e893b1cf4a1'
  Running command git fetch -q https://github.com/dask/dask.git 0d8e12be4c2261b3457978c16aba7e893b1cf4a1


  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git checkout -q 0d8e12be4c2261b3457978c16aba7e893b1cf4a1


  Created wheel for distributed: filename=distributed-2022.12.0+17.gf8302593-py3-none-any.whl size=930327 sha256=abb1ed30352433debceba3bc88253e9d39968b4cb6be16c0c1efdbf36c23a5a6
  Stored in directory: /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-ephem-wheel-cache-q3i4nr3r/wheels/fd/af/11/58dd2291a58d74b51b5fd96dedfb871cc8bba17bef3e91f0b7
Successfully built distributed
  Resolved https://github.com/dask/dask.git to commit 0d8e12be4c2261b3457978c16aba7e893b1cf4a1
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for dask: filename=dask-2022.12.0+13.g0d8e12be-py3-none-any.whl size=1142848 sha256=ee5044daecd9c6a0821eabbe497ce3a92399c83f7a2149a7799069c2659c2350
  Stored in directory: 

╭─────────────────────────────────────── Package Sync Notes ───────────────────────────────────────╮
│                  ╷                                                                               │
│   Package        │ Note                                                                          │
│ ╶────────────────┼─────────────────────────────────────────────────────────────────────────────╴ │
│   openssl        │ Package ignored, no risk                                                      │
│   coiled-runtime │ Package ignored, no risk                                                      │
│   dask           │ Wheel built from                                                              │
│                  │ git+https://github.com/dask/dask.git@0d8e12be4c2261b3457978c16aba7e893b1cf…   │
│   distributed    │ Wheel built from                                                              │
│                  │ git+https://github.com/dask/distributed.git@f83025935383d13033fe0dcd4af2ee…   │
│                  ╵                                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Connection method: Cluster object,Cluster type: coiled.ClusterBeta
Dashboard: http://18.207.166.9:8787,
Dashboard: http://18.207.166.9:8787,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tls://10.0.17.27:8786,Workers: 0
Dashboard: http://10.0.17.27:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [2]:
df = dd.read_parquet("s3://nyc-tlc/trip data/fhvhv_tripdata_*.parquet") 

In [3]:
yes_no = pd.CategoricalDtype(categories=["Y", "N"])
conversions = {}
for column, dtype in df.dtypes.items():
    if dtype == "float64":
        conversions[column] = "float32"
    if dtype == "int64":
        conversions[column] = "int32"
    if dtype == "object":
        conversions[column] = "string[pyarrow]"
    if "flag" in column:
        conversions[column] = yes_no
        
conversions

{'hvfhs_license_num': 'string[pyarrow]',
 'dispatching_base_num': 'string[pyarrow]',
 'originating_base_num': 'string[pyarrow]',
 'PULocationID': 'int32',
 'DOLocationID': 'int32',
 'trip_miles': 'float32',
 'trip_time': 'int32',
 'base_passenger_fare': 'float32',
 'tolls': 'float32',
 'bcf': 'float32',
 'sales_tax': 'float32',
 'congestion_surcharge': 'float32',
 'airport_fee': 'string[pyarrow]',
 'tips': 'float32',
 'driver_pay': 'float32',
 'shared_request_flag': CategoricalDtype(categories=['Y', 'N'], ordered=False),
 'shared_match_flag': CategoricalDtype(categories=['Y', 'N'], ordered=False),
 'access_a_ride_flag': CategoricalDtype(categories=['Y', 'N'], ordered=False),
 'wav_request_flag': CategoricalDtype(categories=['Y', 'N'], ordered=False),
 'wav_match_flag': CategoricalDtype(categories=['Y', 'N'], ordered=False)}

In [ ]:
df = (
    df.astype(conversions)
      .set_index("request_datetime")
      .persist()
)

df.dtypes

In [ ]:
divisions = df.divisions
def part_name(index):
    return str(divisions[index].date()) + ".parquet"
    
part_name(0)

In [ ]:
df = df.repartition(freq="1d").persist()
df.to_parquet(
    "s3://dask-engineering-example-pipelines/xgboost-optuna/nyc-taxi-data",
    name_function=part_name
)

In [ ]:
format_bytes(dd.read_parquet(
    "s3://dask-engineering-example-pipelines/xgboost-optuna/nyc-taxi-data/*.parquet",
    use_nullable_dtypes=True
).memory_usage(deep=True).compute().sum())

In [ ]:
clean.head().dtypes

In [ ]:
s3://coiled-datasets/uber-lyft-tlc/

In [ ]:
format_bytes(dd.read_parquet(
    "s3://coiled-datasets/uber-lyft-tlc/*.parquet",
    use_nullable_dtypes=True
).memory_usage(deep=True).compute().sum())

In [ ]:
format_bytes(dd.read_parquet(
    "s3://coiled-datasets/uber-lyft-tlc/*.parquet",
    use_nullable_dtypes=True
).astype({
    "hvfhs_license_num": "string[pyarrow]", 
    "dispatching_base_num": "string[pyarrow]",
    "originating_base_num": "string[pyarrow]",
    "airport_fee": "string[pyarrow]"
}).persist().memory_usage(deep=True).compute().sum())



### airport data should probably be `float` 

In [ ]:
df.airport_fee.astype(float).unique().compute()